In [1]:
import pandas as pd
import numpy as np
import torch
from src import Utils, LSTM
from skorch.dataset import ValidSplit
from torch.nn import BCEWithLogitsLoss, NLLLoss, BCELoss

/tmp/ipykernel_30822/21230329.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_md' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)
/home/nate/miniconda

In [2]:

torch.cuda.is_available = lambda: False
MODEL = 'model_bert/mBERT-sam.pkl'

TEST_SIZE = 0.4

In [3]:
dataset = Utils.read_csv_file('datasets/datasetall.csv')
dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


In [4]:
Utils.seed_random_number_generators()

Random number generators seeded.


In [5]:
X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

In [6]:
X_train

0        Ok so remember this day you have taken your ph...
1           Freakin mar roxas on the move ?? Election 2016
2        Tbh ang tinuod na lutang kay si Faisal Mangondato
3        petition NOT to cancel the debate between BBM ...
4        Hindi niyo alam kung ga'no ako kasaya na wala ...
                               ...                        
17072    Binaylibre gamot? Libre pahospital? Tapos no i...
17073    Ohfast talk daw! Karen: Lights onlights off? B...
17074    PJ [USERNAME] Dec The mental gymnastics of lit...
17075    [USERNAME]should provide an interpreter for Mr...
17076                                Uunlad tayo kay Binay
Name: text, Length: 17077, dtype: object

In [7]:
y_train

0        1
1        1
2        0
3        0
4        0
        ..
17072    0
17073    1
17074    1
17075    1
17076    0
Name: label, Length: 17077, dtype: int64

In [8]:
X_test

0        Cen Vierneza [USERNAME]"This is not the right ...
1        Kulay Rosas Ang BukasLeni Liwanag Sa DilimLet ...
2        MANALO MAN O MATALO HINDINGHINDI KO IKAKAHIYA ...
3        Minsan nadali na ako nung Ludy’s na peanut but...
4        Kage [USERNAME]Yang mga bicolano na nag susupo...
                               ...                        
11379    Baligtarin natin Kabayan ang survey. Sino sa m...
11380    mahilig silang magpaantay for 25 mins for a ce...
11381    Don't tell me na planado yung mga debates caus...
11382    Louie [USERNAME][USERNAME] never Again Marcos ...
11383    Wow! Baguio Is PinkI can see V sign! The Winni...
Name: text, Length: 11384, dtype: object

In [9]:
y_test

0        1
1        0
2        0
3        0
4        1
        ..
11379    1
11380    1
11381    1
11382    1
11383    0
Name: label, Length: 11384, dtype: int64

In [10]:
VAL_SPLIT = 0.5

X_val, X_test, y_val, y_test = Utils.get_train_test_split(
  pd.DataFrame({
    'text': X_test,
    'label': y_test,
  }), 
  VAL_SPLIT,
)

In [11]:
X_val

0       Choosing the most qualified leaders for countr...
1          Nakakainis si Binay hahahahaha 1 Mister5 Misis
2       K. :) [USERNAME] After Philstar here's ANC spi...
3       [USERNAME] Sa totoo lang mas trip ko na si Fai...
4       Grabe ang daming ads ni BinayPoeRoxas sa TV at...
                              ...                        
5687                        Mar Roxas. Tarunga bisaya pls
5688    Let leni lead us to death and corruption, usel...
5689       Basa naman! Hindi yun puro panood ng fake news
5690    Kapag si Duterte di nanalo alam na! Nandaya si...
5691    Leni supporters bawal kayo ditokakampink Marco...
Name: text, Length: 5692, dtype: object

In [12]:
y_val

0       0
1       1
2       1
3       0
4       0
       ..
5687    0
5688    1
5689    0
5690    1
5691    0
Name: label, Length: 5692, dtype: int64

In [13]:
y_val.value_counts(ascending=True)

label
0    2823
1    2869
Name: count, dtype: int64

In [14]:
X_test

0       Imagine what will happen to the Philippine med...
1                      Puta nangigil nako ky Mar Roxas ba
2       I am petitioning an EDSA People Power if Mar R...
3       Ang ganda ni VP [USERNAME] sa personal. Bait p...
4       RT [USERNAME]: Kay Mar Roxaswhat you see is wh...
                              ...                        
5687    [USERNAME] My least presidential candidates wo...
5688    May bago na namang tv ad si binay. Yung totoo?...
5689    [USERNAME]be like: "don't be scared homie" Len...
5690    If zero income taxVP Binaywhere will funds com...
5691    Leni Kiko All The Way Ipana10 Natin Para Sa La...
Name: text, Length: 5692, dtype: object

In [15]:
y_test

0       0
1       1
2       1
3       0
4       0
       ..
5687    0
5688    1
5689    0
5690    1
5691    0
Name: label, Length: 5692, dtype: int64

In [16]:
y_test.value_counts(ascending=True)

label
0    2823
1    2869
Name: count, dtype: int64

In [17]:
import joblib
model = joblib.load(MODEL)
model

/home/nate/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/nate/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/nate/miniconda3/lib/python3.9/site-packages/skorch/utils.py:568: DeviceWarning: Requested to load data to CUDA but no CUDA devices are available. Loading on device "cpu

Pipeline(steps=[('tokenizer',
                 HuggingfacePretrainedTokenizer(max_length=255,
                                                tokenizer='bert-base-multilingual-cased')),
                ('bert',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=BertModel(
    (bert): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_e...
                (dense): Linear(in_features=3072, out_features=768, bias=True)
                (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
        )
        (pooler): BertPooler(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (activation): Tanh()
        )
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (classifier): Linear(in_features=768, out_features=2, bias=True)
    )
  ),
))])

In [18]:
accuracy, recall, precision, f1 = Utils.get_prediction_results(
  X_val,
  y_val,
  model,
)

In [ ]:
accuracy, recall, precision, f1 = Utils.get_prediction_results(
  X_test,
  y_test,
  model,
)

Accuracy: 0.8185654008438819
Recall: 0.8967921896792189
Precision: 0.777509068923821
F1-score: 0.832901554404145
